# Final Masters Project

## Name: Sreekanth Palagiri, Student ID: R00184198

## Project Topic: Evaluation of Ensemble Approach for Sentiment Analysis on a Small Dataset

##NoteBook1: Trainer Naive Bayees Regression


### **Mount google drive**

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ls "gdrive/My Drive/Colab Notebooks/Masters Project/"

'Airline Tweets dataset'  'Sentence Polarity Dataset'
 glove.6B.300d.txt	   VMDataset


### **Load Data and Preprocess**

In [3]:
import pandas as pd
import numpy as np

df=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Export_loop-sentiment-pos-neg-train_05112020000000.csv")
print(df.groupby(['label']).size())
df.head()

label
Negative     887
Positive    1013
dtype: int64


,label,text
0,Negative,No one cares about marketing slides - a techni...
1,Positive,Are all three hosts providing storage/capacity...
2,Negative,would loved to had managed to get down to the ...
3,Negative,Vending machine at work is out of Dasani water...
4,Positive,"RT @VMwareEdu: Paul Maritz, CEO and President ..."


**Preprocessor to Remove all special characters except emoticons**

In [4]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[^A-Za-z0-9\']+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

print(df['text'][19])
print(preprocessor(df['text'][19]))

Kristina,  Any updates from your side ? I volunteer for beta test :)  - really need that app running as workaround are driving me nuts ....
kristina any updates from your side i volunteer for beta test really need that app running as workaround are driving me nuts :)


In [5]:
df['text'] = df['text'].apply(preprocessor)

In [6]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stemm(text):
  return ' '.join([stemmer.stem(word) for word in text.split()])

print(stemm(df['text'][19]))

kristina ani updat from your side i volunt for beta test realli need that app run as workaround are drive me nut :)


In [7]:
df['text'] = df['text'].apply(stemm)

### **Label Encode the Sentiment**

In [8]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df['Sentiment']=le.fit_transform(df['label'])
df.head(10)

,label,text,Sentiment
0,Negative,no one care about market slide a technic how t...,0
1,Positive,are all three host provid storag capac or is o...,1
2,Negative,would love to had manag to get down to the cam...,0
3,Negative,vend machin at work is out of dasani water boo,0
4,Positive,rt vmwareedu paul maritz ceo and presid of vmw...,1
5,Positive,had few folk ask if you'r interest johnni ferg...,1
6,Positive,get notifi of the latest vsan patch releas and...,1
7,Negative,end of gener support is 3 12 2020 6 5 and 6 7 ...,0
8,Negative,place 4th in funrun today in the 17 39 age gro...,0
9,Positive,yup guy be current under nda know thi for sure,1


In [9]:
le.classes_

array(['Negative', 'Positive'], dtype=object)

### **Seperate Into Train and Test Sets**

In [10]:
from sklearn.model_selection import train_test_split

df_train, df_test, sentiment_train, sentiment_test = train_test_split(df['text'], df['Sentiment'], 
                                                                      random_state=1, test_size=0.15, 
                                                                      shuffle=False)


print('Length of train set:',len(df_train),'Length of test set:',len(df_test))

Length of train set: 1615 Length of test set: 285


### **Naive Bayees Model**

**Define Simple Tokenizer**

In [11]:
def tokenizer(text):
  return [word for word in text.split()]

**Tokenize Text**

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(strip_accents='ascii',
                        lowercase=True,
                        preprocessor=None,
                        tokenizer= tokenizer
                       )
                            
vec.fit(df['text'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents='ascii', token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenizer at 0x7fd87a8ffb00>,
                vocabulary=None)

In [13]:
X_train=vec.transform(df_train)
Y_train=sentiment_train
X_test=vec.transform(df_test)
Y_test=sentiment_test

**Fit Naive Bayees Regressor**

In [14]:
from sklearn.naive_bayes import MultinomialNB

clf= MultinomialNB().fit(X_train, Y_train)
clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

**Train and Test Scores**

In [15]:
print('Train Accuracy Score:',clf.score(X_train, Y_train))
print('Test Accuracy Score:',clf.score(X_test, Y_test))

Train Accuracy Score: 0.951702786377709
Test Accuracy Score: 0.7298245614035088


In [17]:
from sklearn import metrics

Y_pred=clf.predict(X_test)
print('F1 Score:',metrics.f1_score(Y_test,Y_pred),
      'Precision:',metrics.precision_score(Y_test,Y_pred),
      'Recall:',metrics.recall_score(Y_test,Y_pred),
      'Accuracy:',metrics.accuracy_score(Y_test,Y_pred))

F1 Score: 0.763076923076923 Precision: 0.7337278106508875 Recall: 0.7948717948717948 Accuracy: 0.7298245614035088


### **Save the Model**

In [16]:
from joblib import dump

dump(clf, '/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/clf_naivebayees.joblib') 
dump(vec.vocabulary_, '/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/vec_naivebayees.joblib') 

['/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/vec_naivebayees.joblib']